# Panel Plots
We are creating this notebook in order to practice creating panel plots
for Savitzky smoothing method and see how different parameters affect our estimates.

In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import scipy.signal
import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
# start_time = time.time()

In [2]:
# search path for modules
# look @ https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path

####################################################################################
###
###                      Local
###
####################################################################################

################
###
### Core path
###

sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')

################
###
### Directories
###
data_dir = "/Users/hn/Documents/01_research_data" + \
           "/remote_sensing/01_NDVI_TS/00_Eastern_WA_withYear/"

param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"
####################################################################################
###
###                      Aeolus Core path
###
####################################################################################

# sys.path.append('/home/hnoorazar/remote_sensing_codes/')

####################################################################################
###
###                   Aeolus Directories
###
####################################################################################

# data_dir = "/data/hydro/users/Hossein/remote_sensing/01_NDVI_TS/00_Eastern_WA_EE/"
# param_dir = "/home/hnoorazar/remote_sensing_codes/parameters/"

####################################################################################
###
###                   Import remote cores
###
####################################################################################

import remote_sensing_core as rc
import remote_sensing_plot_core as rcp


In [3]:
####################################################################################
###
###      Parameters                   
###
####################################################################################

eleven_colors = ["gray", "lightcoral", "red", "peru",
                 "darkorange", "gold", "olive", "green",
                 "blue", "violet", "deepskyblue"]

Sav_win_size = 9
sav_order = 1
delt = 0.1
irrigated_only = 0
SF_year = 2017
indeks = "EVI"
# we are creating panels where each panel
# consist of different parameters of Savitzky, 
# So, we do not need the following two

# Sav_win_size = int(sys.argv[1]) 
# sav_order = int(sys.argv[2])

# delt = float(sys.argv[1])
# indeks = sys.argv[2]
# irrigated_only = int(sys.argv[3])
# SF_year = int(sys.argv[4])

print ("delta = {fileShape}".format(fileShape=delt))

delta = 0.1


# Read the Data

In [4]:
f_name = "Eastern_WA_" + str(SF_year) + "_70cloud_selectors.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

In [5]:
####################################################################################
###
###                   process data
###
####################################################################################

##################################################################
##################################################################
####
####  plots has to be exact. So, we need 
#### to filter out NASS, and filter by last survey date
####
##################################################################
##################################################################

a_df = a_df[a_df['county']== "Grant"] # Filter Grant
a_df.head(2)

a_df = rc.filter_out_NASS(a_df) # Toss NASS
a_df.head(2)

a_df = rc.filter_by_lastSurvey(a_df, year = SF_year) # filter by last survey date
a_df['SF_year'] = SF_year
a_df.head(2)

########################
######################## Do this for now, till you learn
######################## how to plot 2 years where x is DoY 
######################## 

a_df = a_df[a_df['image_year'] == SF_year]
a_df.head(2)

########################
########################

if irrigated_only == True:
    a_df = rc.filter_out_nonIrrigated(a_df)
    output_Irr = "irrigated_only"
else:
    output_Irr = "non_irrigated_only"
    a_df = rc.filter_out_Irrigated(a_df)
    
a_df.head(2)

######################

# The following columns do not exist in the old data
#
if not('DataSrc' in a_df.columns):
    print ("Data source is being set to NA")
    a_df['DataSrc'] = "NA"
    


if not('CovrCrp' in a_df.columns):
    print ("CovrCrp is being set to NA")
    a_df['CovrCrp'] = "NA"

if (indeks == "EVI"):
    a_df = rc.initial_clean_EVI(a_df)
else:
    a_df = rc.initial_clean_NDVI(a_df)

a_df.head(2)
an_EE_TS = a_df.copy()

,ID,Acres,BSI,county,CropGrp,CropTyp,DataSrc,doy,EVI,ExctAcr,...,NDWI,Notes,PSRI,RtCrpTy,Shap_Ar,Shp_Lng,system_start_time,TRS,image_year,B8
0,131363_WSDA_SF_2017,3.0,NaN,Franklin,Other,developed,wsda,NaN,NaN,2.707829,...,NaN,houses,NaN,NaN,10958.196265,534.903466,NaN,T09R29E8,NaN,NaN
1,138107_WSDA_SF_2017,23.0,NaN,Franklin,Other,developed,wsda,NaN,NaN,23.202388,...,NaN,houses,NaN,NaN,93896.733256,1587.732303,NaN,T09R29E8,NaN,NaN


In [ ]:
### List of unique polygons
polygon_list = an_EE_TS['ID'].unique()
print(len(polygon_list))

In [ ]:
counter = 0

In [ ]:
# for loop here
a_poly = polygon_list[counter]
if (counter%1000 == 0):
    print (counter)
counter += 1
curr_field = an_EE_TS[an_EE_TS['ID']==a_poly].copy()
curr_field.sort_values(by=['image_year', 'doy'], inplace=True)
curr_field.head(2)

################################################################
plant = curr_field['CropTyp'].unique()[0]

# Take care of names, replace "/" and "," and " " by "_"
plant = plant.replace("/", "_")
plant = plant.replace(",", "_")
plant = plant.replace(" ", "_")
plant = plant.replace("__", "_")

county = curr_field['county'].unique()[0]
ID = curr_field['ID'].unique()[0]



In [ ]:
plt.figure();
preds_df.plot();

In [ ]:
# ax.set_title(plot_title);
# ax.set(xlabel='DoY', ylabel=indeks)
# ax.legend(loc="best");
# fig

In [ ]:
fig_name = "/Users/hn/Desktop/" + county + "_" + plant + "_" + str(SF_year) + "_" + str(counter) + '.png'

In [ ]:
plt.show()
plt.savefig(fname = fig_name, \
                    dpi=300,
                    bbox_inches='tight')
plt.close()

In [ ]:
F_1 = rcp.subplots_savitzky(current_field = curr_field, idx= indeks , deltA = 0.1 )

In [ ]:
fig, axs = plt.subplots(2, 2)
axs[0, 0] = F_1
axs[0, 1] = F_1
axs[1, 0] = F_1
axs[1, 1] = F_1

In [ ]:
fig = plt.figure()

fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(2, 2, 1)

ax = fig.add_subplot(2, 2, 2)

ax = fig.add_subplot(2, 2, 3)

ax = fig.add_subplot(2, 2, 4)


In [ ]:
type(F_1)

In [ ]:
# fig, axes = plt.subplots(ncols=7, nrows=4)

# for i, ax in enumerate(axes.flatten()):
#     x = np.random.randint(-5, 5, 20)
#     y = np.random.randint(-5, 5, 20)
#     ax.scatter(x, y)
#     ax.set_title('Axis {}'.format(i))

# plt.tight_layout()

In [ ]:
f2_ax1

In [ ]:
spec2[0, 0]

In [ ]:
def compare(data, fig, rows, cols ):
    for i in range (0,len(data)):
        plt.figure(fig)
        plt.subplot(rows, cols, i+1)
        plt.plot(data[i])
    return

d1 = [1, 2, 3, 4]
d2 = [4, 3, 2, 1]
d3 = [4, 3, 3, 1]
d4 = [3, 4, 1, 2]
data = [d2,d1,d4]

# compare 4 horizontally
compare([d1, d2, d3, d4], fig=1, rows=2, cols=2)

# # compare 4 vertically
# compare([d1, d2, d3, d4], fig=2, rows=4, cols=1)
# # compare 2 vertically
# compare([d2, d3], fig=3, rows=2, cols=1)
# # compare 3 horizontally
# compare([d1, d2, d4], fig=4, rows=1, cols=3)
# # compare 3 vertically
# compare(data, fig=5, rows=3, cols=1)
# plt.tight_layout()
# plt.show()

In [ ]:
figure, axes = plt.subplots(nrows=2, ncols=2)

axes[0, 0].plot(x, y)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(2,2,1)
ax = fig.add_subplot(2,2,2)
ax = fig.add_subplot(2,2,3)

In [ ]:
import matplotlib
rectt = 10, 10, 10, 10
matplotlib.axes.Axes(fig = F_1, rect = rectt )


In [ ]:
fig = plt.figure()
fig.add_subplot(241)
fig.add_subplot(242)
ax = fig.add_subplot(223)
ax.set_title("subplots")

In [ ]:
import matplotlib.gridspec as gridspec
fig2 = plt.figure(constrained_layout=True)
spec2 = gridspec.GridSpec(ncols=2, nrows=2, figure=fig2)

f2_ax1 = fig2.add_subplot(spec2[0, 0])
f2_ax2 = fig2.add_subplot(spec2[0, 1])
f2_ax3 = fig2.add_subplot(spec2[1, 0])
f2_ax4 = fig2.add_subplot(spec2[1, 1])

In [ ]:
fig = plt.figure(constrained_layout=True)
gs = gridspec.GridSpec(2, 3, figure=fig)
ax = fig.add_subplot(gs[0, 0])
ax.add_image(F_1)

In [ ]:
type(F_1)

In [ ]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("outputFuck.pdf")
for countt in range(1, 4):
    pdf.savefig( F_1 )
pdf.close()

In [ ]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("outputFuckMe.pdf")
pdf.savefig( F_1 )
pdf.savefig( F_1 )
pdf.savefig( F_1 )
pdf.savefig( F_1 )
pdf.close()

In [ ]:
fig_name = plot_path + county + "_" + plant + "_SF_year_" + str(SF_year) + "_" + str(counter) + '.pdf'
pdf = matplotlib.backends.backend_pdf.PdfPages(fig_name)
pdf.savefig( F_1 )
pdf.savefig( F_1 )
pdf.savefig( F_1 )
pdf.savefig( F_1 )
pdf.close() 

In [ ]:
plant